In [1]:
# Imports
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
from os.path import join
import pandas as pd
from PIL import Image
import re
import rioxarray
import subprocess
import xarray as xr

# Silence warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [14]:
# CRS Info
# TODO Add docstrings
class EASE_Parameters(object):
    
    def __init__(self, grid='EASE-Grid', hemisphere='Northern'):

        '''
        Retrieve CRS information from URL.
        '''
        self.grid = grid
        self.hemisphere = hemisphere
        self.url = "https://nsidc.org/ease/ease-grid-projection-gt"
        self._table_number = -2
        
    @property
    def table_number(self):
        return self._table_number
    
    @table_number.setter
    def table_number(self, n):
        self._table_number = n
    
    @property
    def table(self):
        df = pd.read_html(self.url)[self._table_number]
        # The Grid Name column name varies over tables, use column number
        return df[(df[df.columns[0]]==self.grid) & (df['Projection'].str.contains(self.hemisphere))]
    
    @property
    def epsg(self):
        '''
        Return the EPSG code corresponding to the CRS. 
        '''
        proj = self.table['Projection'].item()
        pattern = re.compile('EPSG: \d{4}')
        epsg = re.search(pattern, proj)[0]
        return epsg.replace(' ', '')
    
    @property
    def num_cols(self):
        return self.table['Number of Columns'].item()

    @property
    def num_rows(self):
        return self.table['Number of Rows'].item()
    
    @property
    def grid_size(self):
        '''
        Grid cell area is reported as a string {number} {unit} x {number} {unit}.
        '''
        return float(self.table['Grid Cell Area'].item().split(' ')[0].replace(',',''))
    
    @property
    def ulx(self):
        try:
            return self.table['x-axis map coordinate of the outer edge of the upper-left pixel'].item()
        except KeyError:
            return self.table[self.table.columns[-2]].item()
                
    @property
    def uly(self):
        try:
            return self.table['y-axis map coordinate of the outer edge of the upper-left pixel'].item()
        except KeyError:
            return self.table[self.table.columns[-1]].item()
    
    @property
    def lrx(self):
        return self.ulx + self.grid_size*self.num_cols
    
    @property 
    def lry(self):
        return self.uly - self.grid_size*self.num_rows
    

# File conversion function 
def translate_globsnow(directory):
    '''
    Add meta data to the raw file and convert to gdal virtual format.
    '''
    # Obtain CRS information
    param = EASE_Parameters()
    epsg = param.epsg
    ulx, uly, lrx, lry = param.ulx, param.uly, param.lrx, param.lry
    
    # Find files that match this pattern
    pattern = re.compile("(GlobSnow_SWE_L3B_monthly_(\d{4})(\d{2})_v2.0.hdf)")
    for f in glob(join(directory, '*.hdf')):
        match = re.search(pattern, f)
        if match is not None:
            # Create output filename from input filename
            input_file, year, month = match.groups()
            output_file = f"GlobSnow_SWE_Average_{year}_{month}.vrt"
            bash_cmd = (
                f"gdal_translate -of VRT -a_nodata -1 "
                f"-a_srs {epsg} -a_ullr {ulx} {uly} {lrx} {lry} "
                f"HDF4_SDS:UNKNOWN:\"{input_file}\":0 {output_file}"
            )
            subprocess.Popen(bash_cmd, cwd=directory, shell=True, executable='/bin/bash')

def convert_globsnow(directory):
    '''
    Convert .vrt to a .tif file.
    '''
    pattern = re.compile("GlobSnow_SWE_Average_\d{4}_\d{2}.vrt")
    for f in glob(join(directory, '*.vrt')):
        match = re.search(pattern, f)
        if match is not None:
            input_file = match[0]
            output_file = input_file.replace('.vrt', '.tif')
            bash_cmd = (
                f"gdalwarp -of GTiff "
                f"-t_srs EPSG:4326 -r cubic "
                f"{input_file} {output_file}"
            )
            subprocess.Popen(bash_cmd, cwd=directory, shell=True, executable='/bin/bash')
     
    
def plot_tiff(filename):
    '''
    Plot image.
    '''
    im = Image.open(filename)
    plt.imshow(im)

In [15]:
# Run code 
translate_globsnow("../data")
convert_globsnow("../data")